In [ ]:
from pydantic import BaseModel, validator
import re
# tom komment
class CustomerRecord(BaseModel):
    Customer: str
    Address: str
    Phone: str
    Personnummer: str
    BankAccount: str

    @validator("Personnummer")
    def validate_personnummer(cls, v):
        if not re.match(r"^\d{6}-\d{4}$", v):
            raise ValueError("incorrect format personnummer")
        return v

    @validator("BankAccount")
    def validate_iban(cls, v):
        if not v.startswith("SE") or len(v) < 20:
            raise ValueError("incorrect bank account")
        return v
import pandas as pd
from pydantic import ValidationError

df = pd.read_csv("data/sebank_customers_with_accounts.csv")
df = df.rename(columns={"pythCustomer": "Customer"})  # важно!

valid_rows = []
invalid_rows = []

for i, row in df.iterrows():
    try:
        entry = CustomerRecord(**row.to_dict())
        valid_rows.append(row)
    except ValidationError as e:
        error_row = row.copy()
        error_row["error"] = str(e)
        invalid_rows.append(error_row)

df_valid = pd.DataFrame(valid_rows)
df_invalid = pd.DataFrame(invalid_rows)

df_valid.to_csv("output/valid_customers.csv", index=False)
df_invalid.to_csv("output/invalid_customers.csv", index=False)


In [ ]:
import os

os.remove("output/valid_customers.csv")
os.remove("output/invalid_customers.csv")